# Replica project

### The data

Stores the final deduplicated datasets and shows how the search can be done with pretrained embeddings

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd

import torchvision.models as models
from tqdm import tqdm
import pickle
from glob import glob
import sys
from os import path


sys.path.insert(0, "../model/")
from utils import *
from model_replica import ReplicaNet


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir = '/mnt/project_replica/datasets/cini/'
wga_dir = '/mnt/project_replica/datasets/'


# path = '../data/original/'
# path_2 = '../data/'
# data_dir = '../data/'
# replica_dir = '../data/'
# wga_dir = '../data/'




2022-06-11 17:33:23.902626: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata['path'] = replica_dir + metadata['Drawer'].astype(str) + '/' + metadata['Drawer'].astype(str) + '_' + metadata['ImageNumber'].astype(str) + '.jpg'
metadata.head(5)

/tmp/ipykernel_166688/30539250.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL,path
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_1.jpg
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_2.jpg
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_3.jpg
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_4.jpg
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_5.jpg


In [3]:
metadata.columns

Index(['Drawer', 'ImageNumber', 'Author', 'Description', 'AuthorOriginal',
       'AuthorULANLabel', 'AuthorULAN', 'AuthorURL', 'AuthorComplemented',
       'AuthorComplement', 'AuthorNeighbour', 'AuthorNeighbourhood',
       'SimpleCollection', 'CiniCollection', 'CiniNumber', 'CiniTime', 'City',
       'Country', 'FondoStamp', 'Institution', 'Reference', 'AuthorGender',
       'AuthorNationality', 'BeginDate', 'EndDate', 'BiographyLabel',
       'AuthorBirth', 'AuthorDeath', 'AuthorBirthLong', 'AuthorBirthLat',
       'AuthorDeathLong', 'AuthorDeathLat', 'AuthorBirthCity',
       'AuthorDeathCity', 'uid', 'CardboardURL', 'ImageURL', 'path'],
      dtype='object')

## Full data

In [5]:
with open(data_dir + 'image_data_2022_04_28.pkl', 'rb') as infile:
    metadata_2 = pickle.load(infile)

metadata_2.head()


,uid,added,source,path,o_uid,o_title,o_date_begin,o_date_end,o_attribution,o_metadata
0,9515070c-b367-4ee5-b7e0-4eb8cacbcae7,2017-07-11 16:42:41.307381+00:00,cini,cini/2A/2A_1.jpg,c37fd54e-be01-495c-b5ac-0ef08a58fb9f,Veduta dell'Etna e di Taormina.\nGuazzo: 36 x ...,1771.0,1794.0,Fondazione Giorgio Cini,"{'City': 'CAEN', 'Author': 'CHATELET Claude Lo..."
1,a91b058d-a3f4-4e14-a67a-8e710754be5d,2017-07-11 16:42:41.445597+00:00,cini,cini/2A/2A_2.jpg,2b05a698-3e3b-4be6-8aaa-c8d20b3217b5,Veduta del teatro di Taormina.\nGuazzo: 350 x ...,1771.0,1794.0,Fondazione Giorgio Cini,"{'City': 'CAEN', 'Author': 'CHATELET Claude Lo..."
2,1160b942-4c34-4b9c-88a8-2c1d255c5961,2017-07-11 16:42:41.543159+00:00,cini,cini/2A/2A_3.jpg,bc615e17-c55e-48f0-bc3e-e01b325b8b7e,Nettuno e Anfitride.\nmm. 75 x 90,1528.0,1563.0,Fondazione Giorgio Cini,"{'City': 'C H A M B E RY', 'Author': 'SCHIAVON..."
3,a003eb3f-2667-4763-b9a1-1c2daae8b5a1,2017-07-11 16:42:41.631198+00:00,cini,cini/2A/2A_4.jpg,7c1484cf-7069-4cdd-bd3d-2a00935d51a8,Veduta di Nepi.\nMatita: 435 x 385\nT.to.,1815.0,1871.0,Fondazione Giorgio Cini,"{'City': 'C H A RTR ES', 'Author': 'BERTIN Edo..."
4,9903179a-30dc-4a7b-8b39-7456d480e2e2,2017-07-11 16:42:41.742387+00:00,cini,cini/2A/2A_5.jpg,d90c949f-9db1-415b-b5b8-1cb5c0f0f46d,Madonna con Bambino in gloria d'Angeli,1574.0,1629.0,Fondazione Giorgio Cini,"{'City': 'CH A N TIL L Y', 'Author': 'ALIENSE ..."


In [6]:
# metadata_2.reset_index()['o_metadata'][0]
metadata_2 = metadata_2[metadata_2['source'] == 'wga']   
# metadata_2.reset_index()['o_metadata'][0]

In [7]:
metadata_2['AuthorOriginal'] = metadata_2['o_metadata'].apply(lambda x: x['AUTHOR'])
metadata_2['AuthorBirth'] = metadata_2['o_metadata'].apply(lambda x: x['BORN-DIED'])
metadata_2['Description'] = metadata_2['o_metadata'].apply(lambda x: x['TITLE'])
metadata_2['BeginDate'] = metadata_2['o_date_begin']
metadata_2['ImageURL'] = metadata_2['o_metadata'].apply(lambda x: x['URL'])
metadata_2['Type'] = metadata_2['o_metadata'].apply(lambda x: x['TYPE'])
metadata_2['Country'] = metadata_2['o_metadata'].apply(lambda x: x['SCHOOL'])


metadata_2['uid'] = metadata_2['uid'].astype(str).apply(lambda x: x.replace('-', ''))
metadata_2['path'] = wga_dir + metadata_2['path']

metadata_2 = metadata_2[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'ImageURL', 'Type', 'Country', 'uid',]]
metadata_2.head()


,AuthorOriginal,AuthorBirth,Description,path,BeginDate,ImageURL,Type,Country,uid
10789,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,/mnt/project_replica/datasets/WGA/images/a/aac...,1598.0,http://www.wga.hu/html/a/aachen/allegory.html,mythological,German,2355b564f41e41a09bf6ae2a503e152a
10790,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Bacchus, Ceres and Cupid",/mnt/project_replica/datasets/WGA/images/a/aac...,1601.0,http://www.wga.hu/html/a/aachen/bacchus.html,mythological,German,d57ed156ca104cbcb52d0ab4f76c84a1
10791,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Joking Couple,/mnt/project_replica/datasets/WGA/images/a/aac...,1601.0,http://www.wga.hu/html/a/aachen/j_couple.html,genre,German,d51bd5c5a7b1440587b437d428e270a6
10792,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Portrait of Emperor Rudolf II,/mnt/project_replica/datasets/WGA/images/a/aac...,1590.0,http://www.wga.hu/html/a/aachen/rudolf2.html,portrait,German,5d7c0617643d4f7eb58cf38df03b8eb2
10793,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,/mnt/project_replica/datasets/WGA/images/a/aac...,1591.0,http://www.wga.hu/html/a/aachen/selfport.html,portrait,German,b4de719d8ca84709a1fffafe7bb89db0


In [8]:
metadata_tot = pd.concat([metadata[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid', 'ImageURL', 'City',
                                    'Country', 'AuthorBirthCity', 'AuthorDeathCity', 'Drawer']], 
                          metadata_2[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid', 'ImageURL', 'Country', 'Type',]]], axis=0)
print(metadata_tot.shape)
metadata_tot.head(1)

(370628, 13)


,AuthorOriginal,AuthorBirth,Description,path,BeginDate,uid,ImageURL,City,Country,AuthorBirthCity,AuthorDeathCity,Drawer,Type
0,"ALLEGRINI, Francesco",1610.0,tca con penna acquarellata 148 x 203 mm,/mnt/project_replica/datasets/cini/1A/1A_1.jpg,1628.0,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ADELAIDE,NaN,NaN,NaN,1A,NaN


In [9]:
# dropping duplicates
metadata_tot = metadata_tot.groupby('uid').first().reset_index()

### Morphograph

In [10]:
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

,uid,img1,img2,type,annotated
0,0f84770be893475581bebd8c0ce95704,4b6e5a18012e4974a43f6f652c9cad78,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:50.997512+00:00
1,5ade49fc2dc649b9947496a41e2decdb,384b9f017d5f43ba9dcfd92f2af830c4,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:51.238615+00:00
2,bf4d99eaf7da4bd486f9007c90c177a5,94fe74bcebf9442fb5cd9a0327590a84,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:23.705228+00:00
3,b29ec3c83c5641f9b6afadb2409b6746,922fe92d61e744b1a9462c848a916bcf,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:24.021703+00:00
4,ab27b55997b94bef9cca122829e9f64d,94fe74bcebf9442fb5cd9a0327590a84,922fe92d61e744b1a9462c848a916bcf,POSITIVE,2017-07-20 12:18:23.873097+00:00


In [11]:
morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
print(morpho_graph_clusters.shape)
morpho_graph_clusters = morpho_graph_clusters.groupby(['img1', 'img2', 'type']).first().reset_index()
print(morpho_graph_clusters.shape)
morpho_graph_clusters.to_csv(data_dir + 'morphograph_clusters.csv', index=False)
morpho_graph_clusters['uid'] = morpho_graph_clusters['uid_connection']
morpho_graph_clusters['annotated'] = morpho_graph_clusters['date']
morpho_graph_clusters = morpho_graph_clusters.drop(columns=['uid_connection', 'date', 'cluster_file', 'cluster'])
morpho_graph_clusters.head()

(113, 7)
(113, 7)


,img1,img2,type,uid,annotated
0,05f5c4e0325f40019eeb0bb542c1f692,14a738ba9b91454a997738105ccf5c9b,POSITIVE,05f5c4e0325f4001997738105ccf5c9b,06-05-2022_19:29:43
1,05f5c4e0325f40019eeb0bb542c1f692,5a889994248f4f67ad62057ceab2297d,NEGATIVE,05f5c4e0325f4001ad62057ceab2297d,06-05-2022_19:29:43
2,0e12e9649d8e47458a7fa9993642d483,a9e52d895b02468cb4e008478b3b32fa,POSITIVE,0e12e9649d8e4745b4e008478b3b32fa,06-05-2022_19:52:02
3,0edd5584b8a248fba0195ce0819b4b55,441ff1c03fac4adc9ad2914112ec198d,POSITIVE,0edd5584b8a248fb9ad2914112ec198d,06-05-2022_19:32:18
4,0edd5584b8a248fba0195ce0819b4b55,61401ad22c6e47d4aed8948625a0efda,POSITIVE,0edd5584b8a248fbaed8948625a0efda,06-05-2022_19:32:18


In [12]:
morpho_graph_complete = pd.concat([morpho_graph_complete, morpho_graph_clusters], axis=0)
morpho_graph_complete.shape

(120776, 5)

In [13]:
morpho_graph_complete['type'].value_counts()

DUPLICATE        114283
POSITIVE           6380
NON-DUPLICATE        60
PROPOSAL             27
NEGATIVE             26
Name: type, dtype: int64

In [14]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120776, (120776, 5))

In [15]:
positives = get_train_test_split(metadata_tot, morpho_graph_complete)

In [16]:
positives.columns

Index(['index', 'uid', 'AuthorOriginal', 'AuthorBirth', 'Description', 'path',
       'BeginDate', 'ImageURL', 'City', 'Country', 'AuthorBirthCity',
       'AuthorDeathCity', 'Drawer', 'Type', 'uid_connection', 'img1', 'img2',
       'type', 'annotated', 'cluster_file', 'cluster', 'set'],
      dtype='object')

In [17]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)

(5053, 22)


In [18]:
positive = positives.groupby('uid').last().reset_index()
positives.set.value_counts()

train    3045
val      1192
test      816
Name: set, dtype: int64

In [19]:
positive.to_csv(data_dir + 'morpho_dataset.csv')

In [24]:
positive.columns

Index(['uid', 'uid_connection', 'index', 'AuthorOriginal', 'AuthorBirth',
       'Description', 'path', 'BeginDate', 'ImageURL', 'City', 'Country',
       'AuthorBirthCity', 'AuthorDeathCity', 'Drawer', 'Type', 'img1', 'img2',
       'type', 'annotated', 'cluster_file', 'cluster', 'set'],
      dtype='object')

## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [20]:
metadata_tot['BeginDate'] = [int(met) if (len(met) == 4) and ('/' not in met) else 0 for met in metadata_tot['BeginDate'].fillna(0).astype(str).str.strip('c. ').str[:-2].str.strip('-, ')]
metadata_tot.shape

(370106, 13)

In [22]:
metadata_tot = metadata_tot[metadata_tot['BeginDate'] > 1400][metadata_tot['BeginDate'] < 1700]
metadata_tot.shape

/tmp/ipykernel_779992/2324671621.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_tot = metadata_tot[metadata_tot['BeginDate'] > 1400][metadata_tot['BeginDate'] < 1700]


(155004, 13)

In [26]:
morpho_graph_complete = morpho_graph_complete.drop(columns=['cluster_file'])

In [27]:
subset_max = remove_duplicates(metadata_tot, morpho_graph_complete)
subset_max.shape

(301371, 20)

In [28]:
subset_max = subset_max.drop(columns=['cluster','index'])

In [29]:
subset_max = subset_max.sort_values('uid_connection').groupby(['AuthorOriginal', 'Description']).first().reset_index()
print(subset_max.shape)

subset_max = subset_max[subset_max['path'].notnull()].groupby('uid').first().reset_index()
subset_max = subset_max[subset_max['Description'].notnull()]
subset_max['uid_connection'] = subset_max['uid_connection'].fillna('Not uid connection')
print(subset_max.shape)

(236780, 18)
(236780, 18)


In [30]:
to_remove = ['tavolo', 'facciata', 'paesaggio', 'esterno', 'pag.', 'soffitto',
             'sedia', 'veduta', 'scorcio', 'capitell', 'architettonic',
             'fontana', 'edifici', 'libr', 'architettonic']
subset_max = subset_max[~subset_max['Description'].str.lower().str.contains(r'|'.join(to_remove))]
subset_max.shape

(213895, 18)

In [31]:
drawers = [str(num)+str(let) for num in range(70, 86) for let in ['A', 'B', 'C']]
subset_max = subset_max[~subset_max['Drawer'].isin(drawers)]
subset_max.shape

(198892, 18)

In [32]:
subset_max[subset_max['path'].str.contains('WGA')].shape

(28476, 18)

In [33]:
subset_max[subset_max['ImageURL'].str.contains('cini')].shape

(170431, 18)

In [30]:
data = pd.concat([subset_max[subset_max['ImageURL'].str.contains('cini')], positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
print(data.shape)

(83131, 22)


In [34]:
data = pd.concat([subset_max, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
print(data.shape)

(203945, 23)


In [35]:
uid2path = {}

for uid, path in tqdm(zip(data['uid'].unique(), data['path'].unique())):
    uid2path[uid] = path

with open(data_dir + 'uid2path.pkl', 'wb') as outfile:
    pickle.dump(uid2path, outfile)

199572it [00:00, 960964.74it/s] 


In [36]:
data.set.value_counts()

train    3045
val      1192
test      816
Name: set, dtype: int64

In [33]:
data.to_csv(data_dir + 'data.csv', index=False)

In [37]:
data.to_csv(data_dir + 'data_all.csv', index=False)

### Subset

In [34]:
sample = pd.read_csv(data_dir + 'dedup_data_sample_wga.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'level_0'])
sample.head(1)

,uid,cluster,index,Author,AuthorBirth,Description,path,BeginDate,uid_connection,img1,img2,type,annotated,set
0,ef24688d56f74848bcec4307b9b31f07,ef24688d56f74848bcec4307b9b31f07,115875,SEC XVII,NaN,Ritratto di una Palettoni.,/mnt/project_replica/datasets/cini/153C/153C_1...,1600.0,Not uid connection,NaN,NaN,NaN,NaN,NaN


In [35]:
in_sample = list(sample.uid.unique())

data_sample = data[data['uid'].isin(in_sample)].drop(columns=['level_0'])
print(data_sample.shape)
data_sample.head(1)

(12675, 21)


,uid,uid_connection,AuthorOriginal,Description,AuthorBirth,path,BeginDate,ImageURL,City,Country,...,AuthorDeathCity,Drawer,Type,img1,img2,type,annotated,index,cluster,set
11,00046fb9752f41e8a74ab3a2b1168039,0dde0c4734c34ad2a91e5470d8cd1295,"GARNIER, Etienne-Barthélemy",Diana and Her Nymphs,"(b. 1759, Paris, d. 1849, Paris)",/mnt/project_replica/datasets/WGA/images/g/gar...,1751.0,http://www.wga.hu/html/g/garnie/diana_ny.html,None,French,...,None,None,mythological,00046fb9752f41e8a74ab3a2b1168039,d00e4250fe934c53b065725419483521,POSITIVE,2016-05-17 16:24:53+00:00,0.0,72.0,val


In [36]:
data_sample_orig = data_sample[~data_sample['annotated'].astype(str).fillna('').str.contains('2022')]
data_sample_orig.shape

(12586, 21)

In [37]:
data_sample_orig.to_csv(data_dir + 'data_sample.csv', index=False)

In [38]:
data_sample.to_csv(data_dir + 'data_sample_0.csv', index=False)